In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
spark=SparkSession.builder.appName('SQL').getOrCreate()

In [5]:
data=[
(1,'Alice'),
(2,'Bob'),
(13,'John'),
(6,'Alex')
]
col=['student_id','student_name']

students=spark.createDataFrame(data,col)
students.show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|        13|        John|
|         6|        Alex|
+----------+------------+



In [8]:
data=[('Math',),('Physics',),('Programming',)]
col=['Subject_name']

subjects=spark.createDataFrame(data,col)
subjects.show()

+------------+
|Subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+



In [10]:
data=[
(1,"Math"),
(1,'Physics'),
(1,'Programming'),
(2,'Programming'),
(1,'Physics'),
(1,'Math'),
(13,'Math'),
(13,'Programming'),
(13,'Physics'),
(2,'Math'),
(1,'Math')
]

columns = ['student_id','subject_name']

examinations = spark.createDataFrame(data,columns)
examinations.show()

+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+



In [27]:
joinDataFrame = (students
                .crossJoin(subjects)
                # .join(examinations, 'student_id','left')
                .orderBy('student_id'))
joinDataFrame.show()

+----------+------------+------------+
|student_id|student_name|Subject_name|
+----------+------------+------------+
|         1|       Alice|     Physics|
|         1|       Alice| Programming|
|         1|       Alice|        Math|
|         2|         Bob| Programming|
|         2|         Bob|        Math|
|         2|         Bob|     Physics|
|         6|        Alex| Programming|
|         6|        Alex|     Physics|
|         6|        Alex|        Math|
|        13|        John| Programming|
|        13|        John|        Math|
|        13|        John|     Physics|
+----------+------------+------------+



In [30]:
joinDataFrame = (students
                .crossJoin(subjects)
                .join(examinations, ((students['student_id'] == examinations['student_id']) & (examinations['subject_name'] == subjects['subject_name'])),'left')
                .groupBy(students['student_id'], examinations['subject_name']).count().withColumnRename('count','attended_exams')
                .select(students['student_id'],students['student_name'],examinations['subject_name'],'attended_exams')
                .orderBy(students['student_id']))
joinDataFrame.show()

+----------+------------+------------+----------+------------+
|student_id|student_name|Subject_name|student_id|subject_name|
+----------+------------+------------+----------+------------+
|         1|       Alice|        Math|         1|        Math|
|         1|       Alice|        Math|         1|        Math|
|         1|       Alice|        Math|         1|        Math|
|         1|       Alice|     Physics|         1|     Physics|
|         1|       Alice|     Physics|         1|     Physics|
|         1|       Alice| Programming|         1| Programming|
|         2|         Bob|        Math|         2|        Math|
|         2|         Bob|     Physics|      NULL|        NULL|
|         2|         Bob| Programming|         2| Programming|
|         6|        Alex|        Math|      NULL|        NULL|
|         6|        Alex|     Physics|      NULL|        NULL|
|         6|        Alex| Programming|      NULL|        NULL|
|        13|        John|        Math|        13|      

In [47]:
joinDataFrame = (students
                .crossJoin(subjects)
                .join(examinations, ((students['student_id'] == examinations['student_id']) & (examinations['subject_name'] == subjects['subject_name'])),'left')
                .groupBy(students['student_id'], students['student_name'],subjects['subject_name']).agg(F.coalesce(F.count(examinations['student_id']),F.lit(0).alias('attended_exam'))))

In [54]:
select_df = joinDataFrame.join(students, joinDataFrame["student_id"] == students["student_id"], how='left')\
                      .join(subjects, joinDataFrame["subject_name"] == subjects["subject_name"], how='left')\
                      .select(joinDataFrame['student_id'], students['student_name'], joinDataFrame['subject_name'], joinDataFrame['coalesce(count(student_id), 0 AS attended_exam)'])\
                      .orderBy(joinDataFrame['student_id'])

# Show the result
select_df.show()

AnalysisException: Column student_name#1 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.